In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wfdb
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import shuffle

In [5]:
# X1 = np.load('Atrial_Fibrillation_ECG_signal_1780_patients_X.npy')
y_data_1 = np.load('Atrial_Fibrillation_ECG_signal_1780_patients_y_data_1.npy')
# X2 = np.load('NORMAL_ECG_signal_5000_patients_X.npy')
y_data_0 = np.load('NORMAL_ECG_signal_5000_patients_y_data_0.npy')

In [6]:
# X = np.concatenate((X1,X2),axis=0)
y_data = np.concatenate((y_data_1,y_data_0),axis=0)

In [ ]:
y_data.shape

In [ ]:
# X, y_data = shuffle(X, y_data, random_state=42)

In [7]:
xnew = []
ynew = []
patient_ids = []
window_size = 1000  

for i in range(y_data.shape[0]):
    for j in range(5):  
#         start = j * window_size
#         end = (j + 1) * window_size
#         xnew.append(X[i, :, start:end])
        ynew.append(y_data[i]) 
        patient_ids.append(i)

# xnew = np.array(xnew)
ynew = np.array(ynew)
patient_ids = np.array(patient_ids)

In [ ]:
from scipy.signal import coherence
from glob import glob
import scipy.io as sio
import scipy.signal as sig
from scipy.signal import coherence, hilbert 

# mae_matrices_list = []
# rmse_matrices_list = []
cross_corr_matrices_list = []
coherence_matrices_list = []
pli_matrices_list = []
plv_matrices_list = []

for patient_data in xnew:
    cross_corr_matrix = np.zeros((12, 12))
#     mae_matrix = np.zeros((12, 12))
#     rmse_matrix = np.zeros((12, 12))
    coherence_matrix = np.zeros((12, 12))
    pli_matrix = np.zeros((12, 12))
    plv_matrix = np.zeros((12, 12))

    
    for i in range(12):
        for j in range(i, 12):
            lead_i = patient_data[i]
            lead_j = patient_data[j]
            
            cross_corr = np.corrcoef(lead_i, lead_j)[0, 1]
            
#             mae = np.mean(np.abs(lead_i - lead_j))
#             rmse = np.sqrt(np.mean((lead_i - lead_j) ** 2))
            
            f, coh = coherence(lead_i, lead_j)  
            coherence_value = np.mean(coh)  # Storing the average coherence value
            
            
            analytic_i = hilbert(lead_i)
            analytic_j = hilbert(lead_j)
            
            phase_i = np.angle(analytic_i)
            phase_j = np.angle(analytic_j)
                    
            phase_diff = phase_i - phase_j
              
            pli = np.abs(np.mean(np.sign(np.sin(phase_diff))))
               
            plv = np.abs(np.mean(np.exp(1j * phase_diff)))
            
#             mae_matrix[i, j] = mae
#             mae_matrix[j, i] = mae
            
#             rmse_matrix[i, j] = rmse
#             rmse_matrix[j, i] = rmse


            cross_corr_matrix[i, j] = cross_corr
            cross_corr_matrix[j, i] = cross_corr
        
            coherence_matrix[i, j] = coherence_value
            coherence_matrix[j, i] = coherence_value    
            
            pli_matrix[i, j] = pli
            pli_matrix[j, i] = pli
            
            plv_matrix[i, j] = plv
            plv_matrix[j, i] = plv
    
    
    cross_corr_matrices_list.append(cross_corr_matrix)  
#     mae_matrices_list.append(mae_matrix)
#     rmse_matrices_list.append(rmse_matrix)  
    coherence_matrices_list.append(coherence_matrix)
    pli_matrices_list.append(pli_matrix)
    plv_matrices_list.append(plv_matrix)
    
def create_K(A_i, B_i, C_i, D_i):
    top_row = np.concatenate((A_i, B_i), axis=1)
    bottom_row = np.concatenate((C_i, D_i), axis=1)
    return np.concatenate((top_row, bottom_row), axis=0)

K_matrices=[]
for i in range(xnew.shape[0]):
    K_i = create_K(cross_corr_matrices_list[i], coherence_matrices_list[i], pli_matrices_list[i], plv_matrices_list[i])
    K_matrices.append(K_i)

In [ ]:
# np.save('cross_corr_matrices_list_1780+5000_windowed5_not_random.npy', cross_corr_matrices_list)
# np.save('coherence_matrices_list_1780+5000_windowed5_not_random.npy', coherence_matrices_list)
# np.save('pli_matrices_list_1780+5000_windowed5_not_random.npy', pli_matrices_list)
# np.save('plv_matrices_list_1780+5000_windowed5_not_random.npy', plv_matrices_list)
# np.save('K_matrices_1780+5000_windowed5_not_random.npy', K_matrices)

In [2]:
cross_corr_matrices_list = np.load('cross_corr_matrices_list_1780+5000_windowed5_not_random.npy', allow_pickle=True)
coherence_matrices_list = np.load('coherence_matrices_list_1780+5000_windowed5_not_random.npy', allow_pickle=True)
pli_matrices_list = np.load('pli_matrices_list_1780+5000_windowed5_not_random.npy', allow_pickle=True)
plv_matrices_list = np.load('plv_matrices_list_1780+5000_windowed5_not_random.npy', allow_pickle=True)
K_matrices = np.load('K_matrices_1780+5000_windowed5_not_random.npy', allow_pickle=True)

In [17]:
t = np.array(K_matrices).reshape(K_matrices.shape[0],-1)
print(t.shape)

(33900, 576)


In [16]:
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, ynew, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    K.clear_session()

    model = Sequential()

    model.add(Conv2D(30, (3, 3), input_shape=(24, 24, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    model.fit(X_train, y_train,
              batch_size=32,
              epochs=3,
              verbose=1,
              validation_split=0.2)

    predictions = model.predict(X_test)

    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/3
678/678 [==============================] - 6s 9ms/step - loss: 0.2734 - accuracy: 0.8801 - val_loss: 0.0972 - val_accuracy: 0.9677
Epoch 2/3
678/678 [==============================] - 6s 9ms/step - loss: 0.1275 - accuracy: 0.9501 - val_loss: 0.2259 - val_accuracy: 0.9226
Epoch 3/3
212/212 [==============================] - 1s 2ms/step
[[994   7]
 [ 16 339]]
Accuracy: 0.9830383480825958
Sensitivity: 0.9549295774647887
Specificity: 0.993006993006993
Epoch 1/3
678/678 [==============================] - 6s 9ms/step - loss: 0.2670 - accuracy: 0.8825 - val_loss: 0.0418 - val_accuracy: 0.9851
Epoch 2/3
678/678 [==============================] - 6s 9ms/step - loss: 0.1281 - accuracy: 0.9507 - val_loss: 0.0380 - val_accuracy: 0.9871
Epoch 3/3
212/212 [==============================] - 0s 2ms/step
[[997   3]
 [ 26 330]]
Accuracy: 0.9786135693215339
Sensitivity: 0.9269662921348315
Specificity: 0.997
Epoch 1/3
678/678 [==============================] - 7s 9ms/step - loss: 0.2877 - accura

In [23]:
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, ynew, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)
    
    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


[[1000    1]
 [  61  294]]
Accuracy: 0.9542772861356932
Sensitivity: 0.828169014084507
Specificity: 0.999000999000999
[[994   6]
 [ 19 337]]
Accuracy: 0.9815634218289085
Sensitivity: 0.9466292134831461
Specificity: 0.994
[[987  13]
 [ 13 343]]
Accuracy: 0.9808259587020649
Sensitivity: 0.9634831460674157
Specificity: 0.987
[[994   6]
 [ 17 339]]
Accuracy: 0.9830383480825958
Sensitivity: 0.952247191011236
Specificity: 0.994
[[987  12]
 [ 11 346]]
Accuracy: 0.9830383480825958
Sensitivity: 0.969187675070028
Specificity: 0.987987987987988
Average Accuracy: 0.9765486725663717
Average Sensitivity: 0.9319432479432666
Average Specificity: 0.9923977973977974


In [4]:
def get_upper_triangular(matrix):
    rows, cols = matrix.shape
    upper_triangular = []

    for i in range(rows):
        for j in range(i + 1, cols):
            upper_triangular.append(matrix[i, j])

    return np.array(upper_triangular)
t2 = []
for i in range(t.shape[0]):
    t2.append(get_upper_triangular(t[i]))
t = np.array(t2)
print(t.shape)

(33900, 276)


In [5]:
xt = np.load('xt_1798+5000_windowed5_10000_kernels_not_random.npy')
ynew = np.load('ynew_1798+5000_windowed5_10000_kernels_not_random.npy')
patient_ids = np.load('patient_ids_1798+5000_windowed5_10000_kernels_not_random.npy')

In [6]:
print(xt.shape)
xt2 = np.concatenate((xt,t), axis=1)
xt = xt2
print(xt.shape)

(33900, 20000)
(33900, 20276)


In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0
sum_f1 = 0

for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
    X_train, X_test = xt[train_idx], xt[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


[[999   2]
 [  8 347]]
Accuracy: 0.9926253687315634
Sensitivity: 0.9774647887323944
Specificity: 0.998001998001998
[[998   2]
 [ 12 344]]
Accuracy: 0.9896755162241888
Sensitivity: 0.9662921348314607
Specificity: 0.998
[[986  14]
 [  1 355]]
Accuracy: 0.9889380530973452
Sensitivity: 0.9971910112359551
Specificity: 0.986
[[983  17]
 [  2 354]]
Accuracy: 0.9859882005899705
Sensitivity: 0.9943820224719101
Specificity: 0.983
[[994   5]
 [  9 348]]
Accuracy: 0.9896755162241888
Sensitivity: 0.9747899159663865
Specificity: 0.994994994994995
Average Accuracy: 0.9893805309734514
Average Sensitivity: 0.9820239746476214
Average Specificity: 0.9919993985993987


In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV

In [ ]:
# rocket=Rocket()
rocket=Rocket(num_kernels=2000)
rocket.fit(xnew)
xt=rocket.transform(xnew)

xt2 = xt
ynew2 = ynew
patient_ids2 = patient_ids

In [ ]:
# np.save('xt_1798+5000_windowed5_2000_kernels_not_random.npy', xt2)
# np.save('ynew_1798+5000_windowed5_2000_kernels_not_random.npy', ynew2)
# np.save('patient_ids_1798+5000_windowed5_2000_kernels_not_random.npy', patient_ids2)

In [ ]:
# xt = np.load('xt_1798+5000_windowed5_2000_kernels_not_random.npy')
# ynew = np.load('ynew_1798+5000_windowed5_2000_kernels_not_random.npy')
# patient_ids = np.load('patient_ids_1798+5000_windowed5_2000_kernels_not_random.npy')

In [ ]:
xt, ynew, patient_ids = shuffle(xt2, ynew2, patient_ids2, random_state=42)

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sktime.transformations.panel.rocket import Rocket
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0
sum_f1 = 0

for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
    X_train, X_test = xt.iloc[train_idx], xt.iloc[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)


average_accuracy = sum_accuracy / 5

print("Average Accuracy:", average_accuracy)



In [ ]:
window = 5
unhealthy = 1780
folds = 4

unhealthy_x = xt2[:unhealthy*window]
healthy_x = xt2[unhealthy*window:]
unhealthy_y = ynew2[:unhealthy*window]
healthy_y = ynew2[unhealthy*window:]

from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf1 = GroupKFold(n_splits=folds)
gkf2 = GroupKFold(n_splits=folds)

sum_accuracy = 0
sum_f1 = 0

for train_idx1, test_idx1 in gkf1.split(xt2[:unhealthy*window], ynew2[:unhealthy*window], groups=patient_ids2[:unhealthy*window]):
    sum_accuracy = 0
    X_train1, X_test1 = xt2.iloc[train_idx1], xt2.iloc[test_idx1]
    y_train1, y_test1 = ynew2[train_idx1], ynew2[test_idx1]

    for train_idx2, test_idx2 in gkf2.split(xt2[unhealthy*window:], ynew2[unhealthy*window:], groups=patient_ids2[unhealthy*window:]):
        X_train2, X_test2 = healthy_x.iloc[train_idx2], healthy_x.iloc[test_idx2]
        y_train2, y_test2 = healthy_y[train_idx2], healthy_y[test_idx2]

        X_train = np.concatenate((X_train1, X_train2), axis=0) 
        X_test = np.concatenate((X_test1, X_test2), axis=0) 

        y_train = np.concatenate((y_train1, y_train2), axis=0) 
        y_test = np.concatenate((y_test1, y_test2), axis=0)
        
        classifier = SGDClassifier()
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        
        y_pred_majority = []
        for i in range(0, y_pred.shape[0], window):
            majority_vote = np.bincount(y_pred[i:i+window]).argmax()
            y_pred_majority.append(majority_vote)

        y_test_majority = []
        for i in range(0, y_test.shape[0], window):
            majority_vote = np.bincount(y_test[i:i+window]).argmax()
            y_test_majority.append(majority_vote)

        accuracy = accuracy_score(y_test_majority, y_pred_majority)
        sum_accuracy += accuracy

        print(confusion_matrix(y_test_majority, y_pred_majority))
        print("Accuracy:", accuracy)

    average_accuracy = sum_accuracy / folds
    print("Average Accuracy:", average_accuracy)


In [ ]:
# window = 8
# unhealthy = 460
# folds = 3

# unhealthy_x = xt[:unhealthy*window]
# healthy_x = xt[unhealthy*window:]
# unhealthy_y = ynew[:unhealthy*window]
# healthy_y = ynew[unhealthy*window:]

# from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
# from sklearn.model_selection import GroupKFold
# from sklearn.linear_model import SGDClassifier
# import numpy as np

# gkf1 = GroupKFold(n_splits=folds)
# gkf2 = GroupKFold(n_splits=folds)

# sum_accuracy = 0
# sum_sensitivity = 0
# sum_specificity = 0
# sum_f1 = 0

# # for train_idx1, test_idx1,train_idx2, test_idx2  in gkf1.split(xt[:800*5], ynew[:800*5], groups=patient_ids[:800*5]),gkf2.split(xt[800*5:], ynew[800*5:], groups=patient_ids[800*5:]):
# for train_idx1, test_idx1  in gkf1.split(xt[:unhealthy*window], ynew[:unhealthy*window], groups=patient_ids[:unhealthy*window]):
#     sum_accuracy = 0
#     X_train1, X_test1 = xt.iloc[train_idx1], xt.iloc[test_idx1]
#     y_train1, y_test1 = ynew[train_idx1], ynew[test_idx1]

#     for train_idx2, test_idx2  in gkf2.split(xt[unhealthy*window:], ynew[unhealthy*window:], groups=patient_ids[unhealthy*window:]):
#         X_train2, X_test2 = healthy_x.iloc[train_idx2], healthy_x.iloc[test_idx2]
#         y_train2, y_test2 = healthy_y[train_idx2], healthy_y[test_idx2]

#         X_train = np.concatenate((X_train1, X_train2), axis=0) 
#         X_test = np.concatenate((X_test1, X_test2), axis=0) 

#         y_train = np.concatenate((y_train1, y_train2), axis=0) 
#         y_test = np.concatenate((y_test1, y_test2), axis=0)
        
#     # for train_idx, test_idx in gkf.split(xt, ynew, groups=patient_ids):
#     #     X_train, X_test = xt.iloc[train_idx], xt.iloc[test_idx]
#     #     y_train, y_test = ynew[train_idx], ynew[test_idx]

#         classifier = SGDClassifier()
#         classifier.fit(X_train, y_train)

#         y_pred = classifier.predict(X_test)
        
#         y_pred_majority = []
#         for i in range(0, y_pred.shape[0], window):
#             majority_vote = np.bincount(y_pred[i:i+window]).argmax()
#             y_pred_majority.append(majority_vote)

#         y_test_majority = []
#         for i in range(0, y_test.shape[0], window):
#             majority_vote = np.bincount(y_test[i:i+window]).argmax()
#             y_test_majority.append(majority_vote)

#         accuracy = accuracy_score(y_test_majority, y_pred_majority)
#         sum_accuracy += accuracy

#     #     tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

#     #     sensitivity = tp / (tp + fn)
#     #     specificity = tn / (tn + fp)
#     #     f1 = f1_score(y_test_majority, y_pred_majority)

#     #     sum_sensitivity += sensitivity
#     #     sum_specificity += specificity
#     #     sum_f1 += f1

#         print(confusion_matrix(y_test_majority, y_pred_majority))
#         print("Accuracy:", accuracy)
#     #     print("Sensitivity:", sensitivity)
#     #     print("Specificity:", specificity)
#     #     print("F1-Score:", f1)
#     #     print()

#     average_accuracy = sum_accuracy / folds
#     # average_sensitivity = sum_sensitivity / 5
#     # average_specificity = sum_specificity / 5
#     # average_f1 = sum_f1 / 5

#     print("Average Accuracy:", average_accuracy)
#     # print("Average Sensitivity:", average_sensitivity)
#     # print("Average Specificity:", average_specificity)
#     # print("Average F1-Score:", average_f1)


In [ ]:
# from sklearn.linear_model import RidgeClassifierCV
# from sktime.transformations.panel.rocket import Rocket
# from sklearn.metrics import accuracy_score
# from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
# from sklearn.metrics import confusion_matrix
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import scale

# X_train, X_test, y_train, y_test = train_test_split(xt, y_data,
#                                                     test_size=0.25,
#                                                     random_state=42)

# classifier=SGDClassifier()
# classifier.fit(X_train,y_train)

# ypred=classifier.predict(X_test)

# print(confusion_matrix(y_test,ypred))
# print(accuracy_score(y_test,ypred))

In [ ]:
import wfdb
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from biosppy.signals import ecg

import wfdb
import numpy as np
import matplotlib.pyplot as plt
import neurokit2 as nk
from scipy.signal import butter, filtfilt, detrend

# record_name = files_unhealthy[0].replace("\n", "")

path = 'C:\\Users\\Pushpam\\Downloads\\'


record_name = 'JS00001'

# # Load the ECG record
# record = wfdb.rdrecord(path + record_name)

# # Extract the ECG signal
# ecg_signal = record.p_signal[:, 5] 

# # Perform baseline wander removal
# filtered_ecg = signal.detrend(ecg_signal)
# filtered_ecg=filtered_ecg[:10000]
# # Plot the ECG signal with detected R-peaks

signal, meta_val = wfdb.rdsamp(path + record_name)
value = signal.T
ecg_signals = value

filtered_ecg =X[700,3,:2000]#(detrend(ecg_signals))[2,:]

# print(filtered_ecg)
plt.figure(figsize=(10, 4))
plt.plot(np.arange(len(filtered_ecg)) / 1000, filtered_ecg, 'b')
# plt.plot(qrs_inds / record.fs, filtered_ecg[qrs_inds], 'ro', markersize=4)
plt.grid(True)
plt.show()